##**Fundamental Pillar**

In [51]:
!pip install praw

In [52]:
import numpy as np
import requests
import pandas as pd
from datetime import datetime
from time import sleep
from transformers import pipeline
from google.colab import drive
import praw
from datetime import datetime, timedelta

In [53]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [149]:
tickers_csv_path = "/content/drive/My Drive/StockDashboard_Automation/ExportToGitHub/Top Ten Tickers.csv"
df_tickers = pd.read_csv(tickers_csv_path)
selected_ticker = df_tickers.iloc[0, 0]
selected_ticker = 'NVDA'

print(f"📌 Selected Ticker: {selected_ticker}")

📌 Selected Ticker: NVDA


In [150]:
import requests
import pandas as pd

API_KEY = "YVDK82NO6C667N90"

all_dataframe_names = []

def fetch_earnings_data(symbol):
    """
    Fetches earnings data (quarterly and annual) from Alpha Vantage.

    Parameters:
        symbol (str): Stock ticker symbol.

    Returns:
        tuple: (annual dataframe, quarterly dataframe)
    """
    url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if "annualEarnings" not in data or "quarterlyEarnings" not in data:
        print(f"Error fetching earnings data for {symbol}")
        return None, None

    df_annual = pd.DataFrame(data["annualEarnings"]).iloc[::-1].reset_index(drop=True)
    df_quarterly = pd.DataFrame(data["quarterlyEarnings"]).iloc[::-1].reset_index(drop=True)

    return df_annual, df_quarterly

def fetch_financial_statement(symbol, statement):
    """
    Fetches a specific financial statement (annual and quarterly) from Alpha Vantage.

    Parameters:
        symbol (str): Stock ticker symbol.
        statement (str): The type of financial statement ('INCOME_STATEMENT', 'BALANCE_SHEET', 'CASH_FLOW').

    Returns:
        tuple: (annual dataframe, quarterly dataframe)
    """
    url = f"https://www.alphavantage.co/query?function={statement}&symbol={symbol}&apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if "annualReports" not in data or "quarterlyReports" not in data:
        print(f"Error fetching {statement} data for {symbol}")
        return None, None

    df_annual = pd.DataFrame(data["annualReports"]).iloc[::-1].reset_index(drop=True)
    df_quarterly = pd.DataFrame(data["quarterlyReports"]).iloc[::-1].reset_index(drop=True)

    return df_annual, df_quarterly

def fetch_all_data(symbol):
    """
    Fetches Income Statement, Balance Sheet, Cash Flow, and Earnings for a given stock.

    Parameters:
        symbol (str): Stock ticker symbol.

    Returns:
        None: Creates unique DataFrame variables and tracks them in a list.
    """
    statements = {
        "INCOME_STATEMENT": "income",
        "BALANCE_SHEET": "balance_sheet",
        "CASH_FLOW": "cash_flow"
    }

    for statement, name in statements.items():
        annual_df, quarterly_df = fetch_financial_statement(symbol, statement)

        if annual_df is not None and quarterly_df is not None:
            annual_var_name = f"Annual_{name}"
            quarterly_var_name = f"Quarterly_{name}"

            globals()[annual_var_name] = annual_df
            globals()[quarterly_var_name] = quarterly_df

            all_dataframe_names.append(annual_var_name)
            all_dataframe_names.append(quarterly_var_name)

    annual_earnings, quarterly_earnings = fetch_earnings_data(symbol)

    if annual_earnings is not None and quarterly_earnings is not None:
        globals()["Annual_earnings"] = annual_earnings
        globals()["Quarterly_earnings"] = quarterly_earnings

        all_dataframe_names.append("Annual_earnings")
        all_dataframe_names.append("Quarterly_earnings")

symbol = selected_ticker  # Replace with desired stock ticker
fetch_all_data(symbol)

print("\nList of all DataFrame names:")
print(all_dataframe_names)


List of all DataFrame names:
['Annual_income', 'Quarterly_income', 'Annual_balance_sheet', 'Quarterly_balance_sheet', 'Annual_cash_flow', 'Quarterly_cash_flow', 'Annual_earnings', 'Quarterly_earnings']


In [151]:
#Quarterly_income = Quarterly_income.dropna(subset=['totalRevenue']).reset_index(drop=True)
Quarterly_income['totalRevenue'] = pd.to_numeric(Quarterly_income['totalRevenue'], errors='coerce')
Quarterly_income['RevenueGrowth'] = Quarterly_income['totalRevenue'].pct_change() * 100
Quarterly_income = Quarterly_income.dropna().reset_index(drop=True)

In [152]:
Quarterly_income['netIncome'] = pd.to_numeric(Quarterly_income['netIncome'], errors='coerce')
Quarterly_income['Profit_Margin'] = (Quarterly_income['netIncome'] / Quarterly_income['totalRevenue']) * 100

In [153]:
Quarterly_income['operatingExpenses']= pd.to_numeric(Quarterly_income['operatingExpenses'], errors='coerce')
Quarterly_income['operatingExpenses'] = (Quarterly_income['operatingExpenses']/Quarterly_income['totalRevenue'])*100

In [154]:
Quarterly_balance_sheet['shortTermDebt']= pd.to_numeric(Quarterly_balance_sheet['shortTermDebt'], errors='coerce')
Quarterly_balance_sheet['longTermDebt']= pd.to_numeric(Quarterly_balance_sheet['longTermDebt'], errors='coerce')
Quarterly_balance_sheet['Total Debt'] = Quarterly_balance_sheet['shortTermDebt']+Quarterly_balance_sheet['longTermDebt']

In [155]:
common_dates = set(Quarterly_income['fiscalDateEnding']) & set(Quarterly_balance_sheet['fiscalDateEnding']) & set(Quarterly_cash_flow['fiscalDateEnding'])

Quarterly_income = Quarterly_income[Quarterly_income['fiscalDateEnding'].isin(common_dates)].reset_index(drop=True)
Quarterly_balance_sheet = Quarterly_balance_sheet[Quarterly_balance_sheet['fiscalDateEnding'].isin(common_dates)].reset_index(drop=True)
Quarterly_cash_flow = Quarterly_cash_flow[Quarterly_cash_flow['fiscalDateEnding'].isin(common_dates)].reset_index(drop=True)


In [156]:
Quarterly_balance_sheet['totalShareholderEquity'] = pd.to_numeric(Quarterly_balance_sheet['totalShareholderEquity'], errors='coerce')
Quarterly_income['roe_series'] = (Quarterly_income['netIncome']/Quarterly_balance_sheet['totalShareholderEquity'])*100

In [157]:
Quarterly_income["Piotroski_Score"] = 0


In [158]:
for i in range(1, len(Quarterly_income)):
    score = 0
    ROA = Quarterly_income.loc[i, "netIncome"]
    prev_ROA = Quarterly_income.loc[i-1, "netIncome"]
    Quarterly_cash_flow["operatingCashflow"] = Quarterly_cash_flow["operatingCashflow"].apply(pd.to_numeric, errors='coerce')
    CFO = Quarterly_cash_flow.loc[i, "operatingCashflow"]
    score += (ROA > 0)  # ROA > 0
    score += (CFO > 0)  # CFO > 0
    score += (ROA > prev_ROA)  # ROA increasing
    score += (CFO > Quarterly_income.loc[i, "netIncome"])
    Quarterly_balance_sheet["longTermDebt"]= Quarterly_balance_sheet["longTermDebt"].apply(pd.to_numeric, errors='coerce')
    Quarterly_balance_sheet["totalAssets"]= Quarterly_balance_sheet["totalAssets"].apply(pd.to_numeric, errors='coerce')
    # Liquidity & Leverage metrics
    leverage = Quarterly_balance_sheet.loc[i, "longTermDebt"] / Quarterly_balance_sheet.loc[i, "totalAssets"]
    prev_leverage = Quarterly_balance_sheet.loc[i-1, "longTermDebt"] / Quarterly_balance_sheet.loc[i-1, "totalAssets"]
    score += (leverage < prev_leverage)  # Leverage decreasing
    Quarterly_balance_sheet["totalCurrentAssets"]= Quarterly_balance_sheet["totalCurrentAssets"].apply(pd.to_numeric, errors='coerce')
    Quarterly_balance_sheet["totalCurrentLiabilities"]= Quarterly_balance_sheet["totalCurrentLiabilities"].apply(pd.to_numeric, errors='coerce')
    current_ratio = Quarterly_balance_sheet.loc[i, "totalCurrentAssets"] / Quarterly_balance_sheet.loc[i, "totalCurrentLiabilities"]
    prev_current_ratio = Quarterly_balance_sheet.loc[i-1, "totalCurrentAssets"] / Quarterly_balance_sheet.loc[i-1, "totalCurrentLiabilities"]
    score += (current_ratio > prev_current_ratio)

    score += (Quarterly_balance_sheet.loc[i, "commonStockSharesOutstanding"] <= Quarterly_balance_sheet.loc[i-1, "commonStockSharesOutstanding"])

    asset_turnover = Quarterly_income.loc[i, "totalRevenue"] / Quarterly_balance_sheet.loc[i, "totalAssets"]
    prev_asset_turnover = Quarterly_income.loc[i-1, "totalRevenue"] / Quarterly_balance_sheet.loc[i-1, "totalAssets"]
    score += (asset_turnover > prev_asset_turnover)

    Quarterly_income.loc[i, "Piotroski_Score"] = score







In [159]:
Quarterly_balance_sheet['totalLiabilities']= Quarterly_balance_sheet["totalLiabilities"].apply(pd.to_numeric, errors='coerce')
Quarterly_balance_sheet['totalShareholderEquity'] = Quarterly_balance_sheet['totalShareholderEquity'].apply(pd.to_numeric, errors='coerce')
Quarterly_balance_sheet['Debt_to_Equity'] = Quarterly_balance_sheet['totalLiabilities']/Quarterly_balance_sheet['totalShareholderEquity']

In [160]:

earliest_balance_date = Quarterly_balance_sheet["fiscalDateEnding"].min() if Quarterly_balance_sheet is not None else None
earliest_income_date = Quarterly_income["fiscalDateEnding"].min() if Quarterly_income is not None else None
earliest_cash_flow_date = Quarterly_cash_flow["fiscalDateEnding"].min() if Quarterly_cash_flow is not None else None

# Get the overall earliest date available across all financial statements
available_dates = [date for date in [earliest_balance_date, earliest_income_date, earliest_cash_flow_date] if date]
start_date = min(available_dates) if available_dates else "2009-01-01"  # Default fallback
Quarterly_earnings=Quarterly_earnings[Quarterly_earnings['fiscalDateEnding']>=start_date].reset_index(drop=True)

#  Final Result Dataframe

In [161]:
# prompt: I want the revenue growth , revenue values adn fiscal Date ending in a new df with the growth score i am caluclating below

# Create a new DataFrame with the desired columns
results_df = pd.DataFrame({
    'Fiscal Date Ending': Quarterly_income['fiscalDateEnding'],
})



In [162]:
from scipy.stats import rankdata
import numpy as np

def score_percentile(values):
    """
    Returns percentile rank scores between 0 and 100, preserving NaN positions.
    """
    values = pd.Series(values)  # Ensure pandas Series
    valid = values.dropna()
    ranks = rankdata(valid, method='average')  # Rank non-NaN values
    percentiles = 100 * (ranks - 1) / (len(valid) - 1)

    # Reinsert NaNs
    scored = pd.Series(np.nan, index=values.index)
    scored[valid.index] = percentiles
    return scored

def score_revenue_growth(values):
    return score_percentile(values)

def score_profit_margin(values):
    return score_percentile(values)


In [163]:
def score_de_ratio(values):
    """
    Score lower debt-to-equity ratios higher using rule-based logic.
    """
    return [
        100 if v <= 0.5 else
        0 if v >= 2.0 else
        100 * (2.0 - v) / (2.0 - 0.5)
        for v in values
    ]

def score_cash_flow(values):
    """
    Score based on Z-score with sigmoid scaling.
    """
    mean = np.nanmean(values)
    std = np.nanstd(values)
    z_scores = [(v - mean) / std if std > 0 else 0 for v in values]
    return [100 / (1 + np.exp(-z)) for z in z_scores]

def score_roe(values):
    """
    ROE capped at 20%, then scaled to 0–100.
    """
    return [100 * min(max(v, 0), 20) / 20 for v in values]

def score_op_exp(values):
    """
    Lower operating expenses are better — inverted min-max scale.
    """
    max_val = np.nanmax(values)
    return [(1 - (v / max_val)) * 100 if max_val > 0 else 0 for v in values]

def score_eps_surprise(actual_eps, estimated_eps):
    """
    Scales EPS surprise between -10% to +10% into 0–100.
    """
    surprises = [(a - e) / e if e != 0 else 0 for a, e in zip(actual_eps, estimated_eps)]
    clamped = [max(min(s, 0.10), -0.10) for s in surprises]
    return [(s + 0.10) / 0.20 * 100 for s in clamped]


In [164]:
df = pd.DataFrame({
    'Fiscal Date Ending': Quarterly_income['fiscalDateEnding'],
    'RevenueGrowth':Quarterly_income["RevenueGrowth"],
    'Profit_Margin':Quarterly_income["Profit_Margin"],
    "DebtToEquity":Quarterly_balance_sheet["Debt_to_Equity"],
    "operatingCashflow":Quarterly_cash_flow['operatingCashflow'],
    "operatingExpenses":Quarterly_income["operatingExpenses"],
    "ROE": Quarterly_income['roe_series'],
     "EPS_Actual" :pd.to_numeric(Quarterly_earnings['reportedEPS'], errors='coerce'),
    "EPS_Estimate":pd.to_numeric(Quarterly_earnings['estimatedEPS'], errors='coerce')

})


In [165]:
df["Revenue Score"] = score_revenue_growth(df["RevenueGrowth"])
df["Profit Margin Score"] = score_profit_margin(df["Profit_Margin"])
df["D/E Score"] = score_de_ratio(df["DebtToEquity"])
df["Cash Flow Score"] = score_cash_flow(df["operatingCashflow"])
df["ROE Score"] = score_roe(df["ROE"])
df["Op Expense Score"] = score_op_exp(df["operatingExpenses"])
df["EPS Surprise Score"] = score_eps_surprise(df["EPS_Actual"], df["EPS_Estimate"])


In [166]:
import numpy as np
import pandas as pd

# Ensure all numeric columns are cleaned up
def clean_numeric(df, cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Clean all relevant financials
Quarterly_income = clean_numeric(Quarterly_income, ["netIncome", "totalRevenue"])
Quarterly_cash_flow = clean_numeric(Quarterly_cash_flow, ["operatingCashflow"])
Quarterly_balance_sheet = clean_numeric(Quarterly_balance_sheet, [
    "longTermDebt", "totalAssets", "totalCurrentAssets",
    "totalCurrentLiabilities", "commonStockSharesOutstanding"
])

# Initialize score list
piotroski_scores = [np.nan]  # First row has no previous comparison

for i in range(1, len(Quarterly_income)):
    score = 0

    # Profitability
    ROA = Quarterly_income.loc[i, "netIncome"]
    prev_ROA = Quarterly_income.loc[i-1, "netIncome"]
    CFO = Quarterly_cash_flow.loc[i, "operatingCashflow"]

    if ROA > 0: score += 1
    if CFO > 0: score += 1
    if ROA > prev_ROA: score += 1
    if CFO > ROA: score += 1

    # Leverage & Liquidity
    try:
        leverage = Quarterly_balance_sheet.loc[i, "longTermDebt"] / Quarterly_balance_sheet.loc[i, "totalAssets"]
        prev_leverage = Quarterly_balance_sheet.loc[i-1, "longTermDebt"] / Quarterly_balance_sheet.loc[i-1, "totalAssets"]
        if leverage < prev_leverage: score += 1
    except:
        pass

    try:
        curr_ratio = Quarterly_balance_sheet.loc[i, "totalCurrentAssets"] / Quarterly_balance_sheet.loc[i, "totalCurrentLiabilities"]
        prev_ratio = Quarterly_balance_sheet.loc[i-1, "totalCurrentAssets"] / Quarterly_balance_sheet.loc[i-1, "totalCurrentLiabilities"]
        if curr_ratio > prev_ratio: score += 1
    except:
        pass

    # Share dilution check
    shares_now = Quarterly_balance_sheet.loc[i, "commonStockSharesOutstanding"]
    shares_prev = Quarterly_balance_sheet.loc[i-1, "commonStockSharesOutstanding"]
    if shares_now <= shares_prev:
        score += 1

    # Efficiency
    try:
        turnover = Quarterly_income.loc[i, "totalRevenue"] / Quarterly_balance_sheet.loc[i, "totalAssets"]
        turnover_prev = Quarterly_income.loc[i-1, "totalRevenue"] / Quarterly_balance_sheet.loc[i-1, "totalAssets"]
        if turnover > turnover_prev: score += 1
    except:
        pass

    piotroski_scores.append(score)

# Add scores to results
Quarterly_income["Piotroski_Score"] = piotroski_scores
results_df["Piotroski_Score"] = Quarterly_income["Piotroski_Score"]


##**Normalize the Pitroski Score**

In [167]:
df["Piotroski_Score_Scaled"] = results_df["Piotroski_Score"] * (100 / 9)


# Forward Fill

In [168]:
df_fundamental = pd.DataFrame(df)

# Convert Fiscal Date Ending to datetime
df_fundamental["Fiscal Date Ending"] = pd.to_datetime(df_fundamental["Fiscal Date Ending"])


all_dates = pd.date_range(start=df_fundamental["Fiscal Date Ending"].min(), end=pd.Timestamp.today(), freq='D')
df_filled = pd.DataFrame(all_dates, columns=["Date"])
df_filled = df_filled.merge(df_fundamental, left_on="Date", right_on="Fiscal Date Ending", how="left").drop(columns=["Fiscal Date Ending"])
df_filled.fillna(method='ffill', inplace=True)

<ipython-input-168-38f95c6b5027>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled.fillna(method='ffill', inplace=True)


# Fetching the OHLC DATA

In [169]:

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey=YRVTL6XMGYSBO2JN&outputsize=full"
response = requests.get(url).json()
time_series = response.get("Time Series (Daily)", {})
stock_data = []
for date, values in time_series.items():
    stock_data.append({
        "Date": pd.to_datetime(date),
        "Open": float(values["1. open"]),
        "High": float(values["2. high"]),
        "Low": float(values["3. low"]),
        "Close": float(values["4. close"])
    })


df_stock=  pd.DataFrame(stock_data)


symbol=selected_ticker


**Reversing the DF since the OHLC DATA is from latest to earliest and fundamental data was earliest to latest**

In [170]:
df_stock = df_stock.iloc[::-1]

In [171]:
df_stock.reset_index(drop= True)

,Date,Open,High,Low,Close
0,1999-11-01,21.75,24.3800,21.75,23.50
1,1999-11-02,23.94,25.1300,23.75,25.00
2,1999-11-03,26.00,28.1300,25.81,27.44
3,1999-11-04,27.75,29.9400,27.75,29.19
4,1999-11-05,30.00,30.2500,26.88,28.25
...,...,...,...,...,...
6410,2025-04-28,109.69,110.3699,106.02,108.73
6411,2025-04-29,107.67,110.1999,107.44,109.02
6412,2025-04-30,104.47,108.9200,104.08,108.92
6413,2025-05-01,113.08,114.9400,111.30,111.61


In [172]:
df_merged = df_stock.merge(df_filled, on="Date", how="left")


In [173]:
fundamental_data_Close = df_merged.dropna()


In [174]:
fundamental_data_Close

,Date,Open,High,Low,Close,RevenueGrowth,Profit_Margin,DebtToEquity,operatingCashflow,operatingExpenses,...,EPS_Actual,EPS_Estimate,Revenue Score,Profit Margin Score,D/E Score,Cash Flow Score,ROE Score,Op Expense Score,EPS Surprise Score,Piotroski_Score_Scaled
1444,2005-08-01,27.220,27.5900,27.04,27.47,-1.547326,12.844721,0.355217,4.734700e+07,24.317864,...,0.003,0.0030,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.000000,33.333333
1445,2005-08-02,27.500,27.9300,27.46,27.84,-1.547326,12.844721,0.355217,4.734700e+07,24.317864,...,0.003,0.0030,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.000000,33.333333
1446,2005-08-03,27.725,28.1000,27.59,27.84,-1.547326,12.844721,0.355217,4.734700e+07,24.317864,...,0.003,0.0030,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.000000,33.333333
1447,2005-08-04,27.630,27.7500,27.49,27.55,-1.547326,12.844721,0.355217,4.734700e+07,24.317864,...,0.003,0.0030,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.000000,33.333333
1448,2005-08-05,27.430,28.3100,27.32,28.25,-1.547326,12.844721,0.355217,4.734700e+07,24.317864,...,0.003,0.0030,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.000000,33.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,2025-04-28,109.690,110.3699,106.02,108.73,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,0.890,0.8459,69.620253,98.734177,100.0,98.257981,100.000000,81.890709,76.066911,66.666667
6411,2025-04-29,107.670,110.1999,107.44,109.02,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,0.890,0.8459,69.620253,98.734177,100.0,98.257981,100.000000,81.890709,76.066911,66.666667
6412,2025-04-30,104.470,108.9200,104.08,108.92,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,0.890,0.8459,69.620253,98.734177,100.0,98.257981,100.000000,81.890709,76.066911,66.666667
6413,2025-05-01,113.080,114.9400,111.30,111.61,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,0.890,0.8459,69.620253,98.734177,100.0,98.257981,100.000000,81.890709,76.066911,66.666667


In [175]:
fundamental_data_Close["Forward_Return_7d"] = fundamental_data_Close["Close"].shift(-7) / fundamental_data_Close["Close"] - 1
fundamental_data_Close["Target_Up"] = (fundamental_data_Close["Forward_Return_7d"] > 0).astype(int)

<ipython-input-175-de270604c155>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Forward_Return_7d"] = fundamental_data_Close["Close"].shift(-7) / fundamental_data_Close["Close"] - 1
<ipython-input-175-de270604c155>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Target_Up"] = (fundamental_data_Close["Forward_Return_7d"] > 0).astype(int)


In [176]:
len(fundamental_data_Close)

4971

In [177]:
fundamental_data_Close.head()

,Date,Open,High,Low,Close,RevenueGrowth,Profit_Margin,DebtToEquity,operatingCashflow,operatingExpenses,...,Revenue Score,Profit Margin Score,D/E Score,Cash Flow Score,ROE Score,Op Expense Score,EPS Surprise Score,Piotroski_Score_Scaled,Forward_Return_7d,Target_Up
1444,2005-08-01,27.220,27.59,27.04,27.47,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.0,33.333333,0.014197,1
1445,2005-08-02,27.500,27.93,27.46,27.84,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.0,33.333333,0.014727,1
1446,2005-08-03,27.725,28.10,27.59,27.84,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.0,33.333333,0.075790,1
1447,2005-08-04,27.630,27.75,27.49,27.55,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.0,33.333333,0.114701,1
1448,2005-08-05,27.430,28.31,27.32,28.25,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,21.518987,30.379747,100.0,39.128944,28.948827,63.061298,50.0,33.333333,0.068673,1


# **Dynamic Weights**

In [178]:
features = [
    "Revenue Score", "Profit Margin Score", "D/E Score",
    "Cash Flow Score", "ROE Score", "Op Expense Score",
    "EPS Surprise Score", "Piotroski_Score_Scaled"
]

X = fundamental_data_Close[features]
y = fundamental_data_Close["Target_Up"]



In [179]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
log_model = LogisticRegression()
log_model.fit(X, y)

log_weights_raw = log_model.coef_[0]
log_weights = log_weights_raw / np.sum(np.abs(log_weights_raw))

fundamental_data_Close["Final_Fundamental_Score_LogReg"] = np.dot(X, log_weights)

# ----------------------------
# Step 3: Random Forest
# ----------------------------

rf_model = RandomForestClassifier()
rf_model.fit(X, y)

rf_importances = rf_model.feature_importances_
rf_weights = rf_importances / rf_importances.sum()

fundamental_data_Close["Final_Fundamental_Score_RF"] = np.dot(X, rf_weights)

# ----------------------------
# Step 4: Rescale to 0–100
# ----------------------------

scaler = MinMaxScaler((0, 100))
fundamental_data_Close["Final_Fundamental_Score_LogReg"] = scaler.fit_transform(
    fundamental_data_Close[["Final_Fundamental_Score_LogReg"]]
)
fundamental_data_Close["Final_Fundamental_Score_RF"] = scaler.fit_transform(
    fundamental_data_Close[["Final_Fundamental_Score_RF"]]
)

# ----------------------------
# Step 5: View Feature Weights
# ----------------------------

weights_df = pd.DataFrame({
    "Feature": features,
    "Logistic_Weight": log_weights,
    "RandomForest_Weight": rf_weights
}).sort_values(by="RandomForest_Weight", ascending=False)

print(weights_df)

<ipython-input-179-1dbf01e5c405>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Final_Fundamental_Score_LogReg"] = np.dot(X, log_weights)


                  Feature  Logistic_Weight  RandomForest_Weight
1     Profit Margin Score         0.095001             0.178767
5        Op Expense Score        -0.357008             0.166008
4               ROE Score        -0.031984             0.156300
0           Revenue Score        -0.015530             0.147388
3         Cash Flow Score         0.113147             0.143833
7  Piotroski_Score_Scaled         0.048494             0.085750
2               D/E Score        -0.328162             0.077546
6      EPS Surprise Score         0.010675             0.044408


<ipython-input-179-1dbf01e5c405>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Final_Fundamental_Score_RF"] = np.dot(X, rf_weights)
<ipython-input-179-1dbf01e5c405>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Final_Fundamental_Score_LogReg"] = scaler.fit_transform(
<ipython-input-179-1dbf01e5c405>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [180]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def correlation_weighted_score(df, feature_cols, target_col, score_col_name="Final_Fundamental_Score_Corr"):
    """
    Calculate a composite score using correlation-based weights.

    Parameters:
    - df: DataFrame containing the scored features and target
    - feature_cols: List of columns representing scored features (0–100 scale)
    - target_col: Column representing future return or binary target
    - score_col_name: Name for the resulting score column

    Returns:
    - df: Original DataFrame with a new column for correlation-based score
    - weights: Dict of feature weights
    """
    # Drop rows with NaNs
    df_clean = df.dropna(subset=feature_cols + [target_col])

    # Calculate absolute correlation of each feature with target
    correlations = df_clean[feature_cols].corrwith(df_clean[target_col])
    abs_corr = correlations.abs()
    weights = abs_corr / abs_corr.sum()

    # Compute weighted score
    df_clean[score_col_name] = np.dot(df_clean[feature_cols], weights)

    # Normalize to 0–100
    scaler = MinMaxScaler((0, 100))
    df_clean[score_col_name] = scaler.fit_transform(df_clean[[score_col_name]])

    # Assign back to original DataFrame
    df.loc[df_clean.index, score_col_name] = df_clean[score_col_name]

    return df, weights.to_dict()


In [181]:
# Define your features and target
fundamental_features = [
    "Revenue Score", "Profit Margin Score", "D/E Score",
    "Cash Flow Score", "ROE Score", "Op Expense Score",
    "EPS Surprise Score", "Piotroski_Score_Scaled"
]

# Target can be a forward return column
target = "Forward_Return_7d"

# Run the function
fundamental_data_Close, feature_weights = correlation_weighted_score(fundamental_data_Close, fundamental_features, target)

# View the weights
print("Correlation-based weights:", feature_weights)


Correlation-based weights: {'Revenue Score': 0.040013321347446694, 'Profit Margin Score': 0.1160531237502596, 'D/E Score': 0.30119785495719253, 'Cash Flow Score': 0.07976370844221412, 'ROE Score': 0.14181669375847983, 'Op Expense Score': 0.2735908683976609, 'EPS Surprise Score': 0.019932679464521326, 'Piotroski_Score_Scaled': 0.027631749882224877}


<ipython-input-180-e74030c7febe>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[score_col_name] = np.dot(df_clean[feature_cols], weights)
<ipython-input-180-e74030c7febe>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[score_col_name] = scaler.fit_transform(df_clean[[score_col_name]])
<ipython-input-180-e74030c7febe>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [182]:
fundamental_data_Close["Final_Fundamental_Score_Corr"] = fundamental_data_Close["Final_Fundamental_Score_Corr"].ffill()


<ipython-input-182-353665b8a2ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Final_Fundamental_Score_Corr"] = fundamental_data_Close["Final_Fundamental_Score_Corr"].ffill()


In [183]:
fundamental_data_Close.head()

,Date,Open,High,Low,Close,RevenueGrowth,Profit_Margin,DebtToEquity,operatingCashflow,operatingExpenses,...,Cash Flow Score,ROE Score,Op Expense Score,EPS Surprise Score,Piotroski_Score_Scaled,Forward_Return_7d,Target_Up,Final_Fundamental_Score_LogReg,Final_Fundamental_Score_RF,Final_Fundamental_Score_Corr
1444,2005-08-01,27.220,27.59,27.04,27.47,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,39.128944,28.948827,63.061298,50.0,33.333333,0.014197,1,0.0,30.102006,43.92369
1445,2005-08-02,27.500,27.93,27.46,27.84,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,39.128944,28.948827,63.061298,50.0,33.333333,0.014727,1,0.0,30.102006,43.92369
1446,2005-08-03,27.725,28.10,27.59,27.84,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,39.128944,28.948827,63.061298,50.0,33.333333,0.075790,1,0.0,30.102006,43.92369
1447,2005-08-04,27.630,27.75,27.49,27.55,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,39.128944,28.948827,63.061298,50.0,33.333333,0.114701,1,0.0,30.102006,43.92369
1448,2005-08-05,27.430,28.31,27.32,28.25,-1.547326,12.844721,0.355217,47347000.0,24.317864,...,39.128944,28.948827,63.061298,50.0,33.333333,0.068673,1,0.0,30.102006,43.92369


In [184]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

def compute_fundamental_scores_threshold_based(df, fund_features, close_col="Close", lookahead=7,
                                               upper_threshold=0.01, lower_threshold=-0.01):
    """
    Computes fundamental scores using logistic regression, correlation, and random forest
    with 3-class target labels (Up, Down, Neutral) based on fixed return thresholds.
    """

    df = df.copy()

    # Step 1: Compute forward return
    df["Forward_Return_Pct"] = df[close_col].shift(-lookahead) / df[close_col] - 1

    # Step 2: Threshold-based labeling
    def label_return(x):
        if x > upper_threshold:
            return 1    # Up
        elif x < lower_threshold:
            return -1   # Down
        else:
            return 0    # Neutral

    df["Target_Label"] = df["Forward_Return_Pct"].apply(label_return)

    # Step 3: Clean data
    df_clean = df.dropna(subset=fund_features + ["Target_Label"])
    X = df_clean[fund_features]
    y = df_clean["Target_Label"]

    # Step 4: Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # -----------------------
    # Logistic Regression (Multiclass)
    # -----------------------
    log_model = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
    log_model.fit(X_train, y_train)
    log_weights = log_model.coef_[0] / sum(abs(log_model.coef_[0]))
    df_clean["Fund_Score_LogReg"] = np.dot(X, log_weights)

    # -----------------------
    # Correlation-Based
    # -----------------------
    y_numeric = y.replace({-1: 0, 0: 0.5, 1: 1})  # Approx numeric scale
    corr = X.corrwith(y_numeric).abs()
    corr_weights = corr / corr.sum()
    df_clean["Fund_Score_Corr"] = np.dot(X, corr_weights)

    # -----------------------
    # Random Forest
    # -----------------------
    rf_model = RandomForestClassifier()
    rf_model.fit(X_train, y_train)
    rf_weights = rf_model.feature_importances_ / rf_model.feature_importances_.sum()
    df_clean["Fund_Score_RF"] = np.dot(X, rf_weights)

    # -----------------------
    # Normalize to 0–100 scale
    # -----------------------
    scaler = MinMaxScaler((0, 100))
    for col in ["Fund_Score_LogReg", "Fund_Score_Corr", "Fund_Score_RF"]:
        df_clean[col] = scaler.fit_transform(df_clean[[col]])
        df.loc[df_clean.index, col] = df_clean[col]

    # -----------------------
    # Evaluation
    # -----------------------
    print("Logistic Regression Classification Report:")
    print(classification_report(y_test, log_model.predict(X_test), zero_division=0))

    print("Random Forest Classification Report:")
    print(classification_report(y_test, rf_model.predict(X_test), zero_division=0))

    # -----------------------
    # Return final scores and weights
    # -----------------------
    weights = {
        "Logistic": dict(zip(fund_features, log_weights)),
        "Correlation": dict(zip(fund_features, corr_weights)),
        "RandomForest": dict(zip(fund_features, rf_weights))
    }

    return df, weights


In [185]:
fundamental_features = [
    "Revenue Score", "Profit Margin Score", "D/E Score",
    "Cash Flow Score", "ROE Score", "Op Expense Score",
    "EPS Surprise Score", "Piotroski_Score_Scaled"
]

df_with_fund_scores, fund_weights = compute_fundamental_scores_threshold_based(
    fundamental_data_Close,
    fundamental_features,
    close_col="Close",
    lookahead= 7,
    upper_threshold=0.015,   # +1% return
    lower_threshold=-0.015   # -1% return
)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Classification Report:
              precision    recall  f1-score   support

          -1       0.40      0.06      0.10       323
           0       0.00      0.00      0.00       185
           1       0.49      0.96      0.65       487

    accuracy                           0.49       995
   macro avg       0.30      0.34      0.25       995
weighted avg       0.37      0.49      0.35       995

Random Forest Classification Report:
              precision    recall  f1-score   support

          -1       0.42      0.44      0.43       323
           0       0.29      0.02      0.04       185
           1       0.54      0.71      0.61       487

    accuracy                           0.50       995
   macro avg       0.42      0.39      0.36       995
weighted avg       0.45      0.50      0.45       995



# **Quantile Calculation for thresholds **

In [186]:
import pandas as pd
import numpy as np

def determine_thresholds_via_quantiles(
    df,
    close_col="Close",
    lookahead=10,
    lower_quantile=0.25,
    upper_quantile=0.75
):
    """
    1) Computes the forward 30-day return.
    2) Determines threshold boundaries for 3-class labeling by
       picking the returns at 'lower_quantile' and 'upper_quantile'.
    3) Returns the two thresholds.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing at least a 'Close' price column.
    close_col : str
        Column name for the asset's close price. Default = 'Close'.
    lookahead : int
        Number of days ahead to compute forward return.
    lower_quantile : float
        Quantile for the DOWN/FLAT boundary.
    upper_quantile : float
        Quantile for the FLAT/UP boundary.

    Returns
    -------
    (float, float)
        A tuple (lower_threshold, upper_threshold).
    """

    data = df.copy()

    # Compute the forward return = (Close[t+lookahead] / Close[t]) - 1
    data["Forward_Return"] = data[close_col].shift(-lookahead) / data[close_col] - 1

    # Drop rows where Forward_Return is NaN (due to shift)
    data.dropna(subset=["Forward_Return"], inplace=True)

    # Get the distribution of forward returns
    lower_threshold = data["Forward_Return"].quantile(lower_quantile)
    upper_threshold = data["Forward_Return"].quantile(upper_quantile)

    return lower_threshold, upper_threshold

def label_3class(x, lower_threshold, upper_threshold):
    """
    Given a forward return x, return:
      -1 if x < lower_threshold  (DOWN)
       0 if lower_threshold <= x <= upper_threshold (FLAT)
       1 if x > upper_threshold  (UP)
    """
    if x < lower_threshold:
        return -1
    elif x > upper_threshold:
        return 1
    else:
        return 0

# Example usage:
if __name__ == "__main__":
    # Suppose df has a Date index and a 'Close' column.
    # For illustration, let's create a simple random 'Close' series:
    df_example = fundamental_data_Close.copy()

    # 1) Determine thresholds for 30-day lookahead using 25th and 75th percentiles
    lower_thr, upper_thr = determine_thresholds_via_quantiles(
        df_example, close_col="Close", lookahead=10, lower_quantile=0.25, upper_quantile=0.75
    )
    print(f"Calculated thresholds:\n  Lower: {lower_thr:.3%}\n  Upper: {upper_thr:.3%}")

    # 2) Label the entire DataFrame using these thresholds
    df_example["Forward_Return"] = df_example["Close"].shift(-30) / df_example["Close"] - 1
    df_example["Label"] = df_example["Forward_Return"].apply(
        lambda x: label_3class(x, lower_thr, upper_thr)
    )

    # Check final distribution of classes (excluding NaN from shift)
    final_df = df_example.dropna(subset=["Forward_Return", "Label"])
    print("\nClass distribution:")
    print(final_df["Label"].value_counts())


Calculated thresholds:
  Lower: -3.804%
  Upper: 7.089%

Class distribution:
Label
 1    2090
-1    1458
 0    1393
Name: count, dtype: int64


In [187]:
from imblearn.over_sampling import SMOTE

In [188]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

def tune_fundamental_rf_asymmetric(
    df,
    fund_features,
    close_col="Close",
    down_thresholds=[0.05],
    up_thresholds=[0.08],
    lookaheads=[5, 7, 10 ,30],
    test_sizes=[0.2],
    rf_param_grid=None,
    random_state=42
):
    if rf_param_grid is None:
        rf_param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [5, 7],
            'min_samples_leaf': [3, 5]
        }

    param_grid = list(ParameterGrid(rf_param_grid))
    results = []

    def label_return_asymmetric(x, d_thr, u_thr):
        if x < -d_thr:
            return -1
        elif x > u_thr:
            return 1
        else:
            return 0

    for down_threshold in down_thresholds:
        for up_threshold in up_thresholds:
            for lookahead in lookaheads:
                for test_size in test_sizes:
                    for params in param_grid:
                        data = df.copy()
                        data["Forward_Return"] = data[close_col].shift(-lookahead) / data[close_col] - 1
                        data["Target_3Class"] = data["Forward_Return"].apply(
                            lambda x: label_return_asymmetric(x, down_threshold, up_threshold)
                        )

                        data_model = data.dropna(subset=fund_features + ["Target_3Class"])
                        # Inside the inner loop, after Target_3Class creation
                        label_counts = data["Target_3Class"].value_counts().to_dict()
                        # print(label_counts)

                        if data_model.empty or data_model["Target_3Class"].nunique() < 2:
                            continue

                        X = data_model[fund_features]
                        y = data_model["Target_3Class"]

                        # try:
                        #     X_resampled, y_resampled = SMOTE(random_state=random_state).fit_resample(X, y)
                        # except ValueError:
                        #     continue

                        X_train, X_test, y_train, y_test = train_test_split(
                            X, y, test_size=test_size, random_state=random_state, stratify= y
                        )

                        rf = RandomForestClassifier(random_state=random_state, class_weight= 'balanced_subsample' , **params)
                        rf.fit(X_train, y_train)

                        y_train_pred = rf.predict(X_train)
                        y_test_pred = rf.predict(X_test)

                        # Get feature importances and score
                        fund_weights = rf.feature_importances_ / np.sum(rf.feature_importances_)
                        fund_score_raw = np.dot(X, fund_weights)
                        fund_score_norm = MinMaxScaler((0, 100)).fit_transform(fund_score_raw.reshape(-1, 1)).flatten()
                        data_model["Fund_Score_RF_Tuned"] = fund_score_norm
                        data.loc[data_model.index, "Fund_Score_RF_Tuned"] = data_model["Fund_Score_RF_Tuned"]



                        # Class-wise metrics
                        labels = [-1, 0, 1]
                        test_f1_pc = f1_score(y_test, y_test_pred, average=None, labels=labels, zero_division=0)
                        test_prec_pc = precision_score(y_test, y_test_pred, average=None, labels=labels, zero_division=0)
                        test_recall_pc = recall_score(y_test, y_test_pred, average=None, labels=labels, zero_division=0)

                        fund_weights = rf.feature_importances_ / np.sum(rf.feature_importances_)
                        results.append({
                            "Down_Threshold": down_threshold,
                            "Up_Threshold": up_threshold,
                            "Lookahead": lookahead,
                            "Test_Size": test_size,
                            "Test_Accuracy": accuracy_score(y_test, y_test_pred),
                            "Train_F1_Macro"  :f1_score(y_train, y_train_pred, average='macro', zero_division=0),
                            "Test_F1_Macro": f1_score(y_test, y_test_pred, average='macro', zero_division=0),
                            "F1_gap": f1_score(y_train, y_train_pred, average='macro', zero_division=0) - f1_score(y_test, y_test_pred, average='macro', zero_division=0),
                            "Test_Precision_Macro": precision_score(y_test, y_test_pred, average='macro', zero_division=0),
                            "Test_Recall_Macro": recall_score(y_test, y_test_pred, average='macro', zero_division=0),

                            "Test_F1_-1": test_f1_pc[0],
                            "Test_F1_0": test_f1_pc[1],
                            "Test_F1_1": test_f1_pc[2],
                            "Test_Precision_-1": test_prec_pc[0],
                            "Test_Precision_0": test_prec_pc[1],
                            "Test_Precision_1": test_prec_pc[2],
                            "Test_Recall_-1": test_recall_pc[0],
                            "Test_Recall_0": test_recall_pc[1],
                            "Test_Recall_1": test_recall_pc[2],

                            "Label_Count_-1": label_counts.get(-1, 0),
                            "Label_Count_0": label_counts.get(0, 0),
                            "Label_Count_1": label_counts.get(1, 0),


                            **params,
                            "Feature_Weights": dict(zip(fund_features, fund_weights)),
                            "Fund_Score_Col": data["Fund_Score_RF_Tuned"] if "Fund_Score_RF_Tuned" in data else None
                        })

    return pd.DataFrame(results)


In [189]:
fundamental_data_Close.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'RevenueGrowth',
       'Profit_Margin', 'DebtToEquity', 'operatingCashflow',
       'operatingExpenses', 'ROE', 'EPS_Actual', 'EPS_Estimate',
       'Revenue Score', 'Profit Margin Score', 'D/E Score', 'Cash Flow Score',
       'ROE Score', 'Op Expense Score', 'EPS Surprise Score',
       'Piotroski_Score_Scaled', 'Forward_Return_7d', 'Target_Up',
       'Final_Fundamental_Score_LogReg', 'Final_Fundamental_Score_RF',
       'Final_Fundamental_Score_Corr'],
      dtype='object')

In [190]:
fund_results_df = tune_fundamental_rf_asymmetric(
    df=fundamental_data_Close,
    fund_features=[
        "Revenue Score", "Profit Margin Score", "D/E Score",
        "Cash Flow Score", "ROE Score", "Op Expense Score",
        "EPS Surprise Score", "Piotroski_Score_Scaled"
    ],
    close_col="Close",
    down_thresholds=[0.05 ,0.08 , 0.10],
    up_thresholds=[0.08,0.10,0.20,0.30],
    lookaheads=[10,30],
    rf_param_grid={
        'n_estimators': [100, 200,300],
        'max_depth': [5, 7],
        'min_samples_leaf': [3, 5,]
    }
)


In [191]:
fund_results_df.columns

Index(['Down_Threshold', 'Up_Threshold', 'Lookahead', 'Test_Size',
       'Test_Accuracy', 'Train_F1_Macro', 'Test_F1_Macro', 'F1_gap',
       'Test_Precision_Macro', 'Test_Recall_Macro', 'Test_F1_-1', 'Test_F1_0',
       'Test_F1_1', 'Test_Precision_-1', 'Test_Precision_0',
       'Test_Precision_1', 'Test_Recall_-1', 'Test_Recall_0', 'Test_Recall_1',
       'Label_Count_-1', 'Label_Count_0', 'Label_Count_1', 'max_depth',
       'min_samples_leaf', 'n_estimators', 'Feature_Weights',
       'Fund_Score_Col'],
      dtype='object')

In [192]:
def tag_fit_quality(row, min_f1=0.55, max_gap=0.10):
    if row["Test_F1_Macro"] < min_f1 and row["Train_F1_Macro"] < min_f1:
        return "Underfitting"
    elif row["F1_gap"] > max_gap:
        return "Overfitting"
    elif row["Test_F1_Macro"] >= min_f1 and row["F1_gap"] <= max_gap:
        return "Good Fit"
    else:
        return "Uncertain"

fund_results_df["Fit_Quality"] = fund_results_df.apply(tag_fit_quality, axis=1)

In [193]:
fund_results_df["Fit_Quality"].value_counts()

,count
Fit_Quality,
Underfitting,144
Good Fit,136
Uncertain,8


In [194]:
perfect_fits = fund_results_df[
    (fund_results_df["Test_F1_Macro"] >= 0.50) &
    (fund_results_df["F1_gap"] <= 0.05) &
    (fund_results_df["Test_Accuracy"] >= 0.55) &
     (fund_results_df["Fit_Quality"] == "Good Fit")&
    (fund_results_df["Label_Count_-1"] >= 500)&
    (fund_results_df["Label_Count_0"] >= 500)&
    (fund_results_df["Label_Count_1"] >= 500)
].sort_values(by="Test_F1_Macro", ascending=False)

In [195]:
perfect_fits.iloc[0]

,215
Down_Threshold,0.1
Up_Threshold,0.08
Lookahead,30
Test_Size,0.2
Test_Accuracy,0.652261
Train_F1_Macro,0.642222
Test_F1_Macro,0.652547
F1_gap,-0.010326
Test_Precision_Macro,0.647768
Test_Recall_Macro,0.69432


In [196]:
perfect_fits['Feature_Weights'].iloc[0]

{'Revenue Score': np.float64(0.14759051197916162),
 'Profit Margin Score': np.float64(0.1651528396179315),
 'D/E Score': np.float64(0.11448042756070197),
 'Cash Flow Score': np.float64(0.1193111511785014),
 'ROE Score': np.float64(0.14174031100111067),
 'Op Expense Score': np.float64(0.18049247977825453),
 'EPS Surprise Score': np.float64(0.06368302362441401),
 'Piotroski_Score_Scaled': np.float64(0.06754925525992436)}

In [197]:
fund_weights = perfect_fits['Feature_Weights'].iloc[0]

In [198]:
fund_weights

{'Revenue Score': np.float64(0.14759051197916162),
 'Profit Margin Score': np.float64(0.1651528396179315),
 'D/E Score': np.float64(0.11448042756070197),
 'Cash Flow Score': np.float64(0.1193111511785014),
 'ROE Score': np.float64(0.14174031100111067),
 'Op Expense Score': np.float64(0.18049247977825453),
 'EPS Surprise Score': np.float64(0.06368302362441401),
 'Piotroski_Score_Scaled': np.float64(0.06754925525992436)}

In [199]:
features = [
    "Revenue Score", "Profit Margin Score", "D/E Score",
    "Cash Flow Score", "ROE Score", "Op Expense Score",
    "EPS Surprise Score", "Piotroski_Score_Scaled"
]

X = fundamental_data_Close[features]
fund_weights_list = [fund_weights[feature] for feature in features]
fundamental_data_Close['Fund_Score_comp'] = np.dot(X,fund_weights_list)

<ipython-input-199-133120c84c76>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close['Fund_Score_comp'] = np.dot(X,fund_weights_list)


In [200]:
fundamental_data_Close.tail()

,Date,Open,High,Low,Close,RevenueGrowth,Profit_Margin,DebtToEquity,operatingCashflow,operatingExpenses,...,ROE Score,Op Expense Score,EPS Surprise Score,Piotroski_Score_Scaled,Forward_Return_7d,Target_Up,Final_Fundamental_Score_LogReg,Final_Fundamental_Score_RF,Final_Fundamental_Score_Corr,Fund_Score_comp
6410,2025-04-28,109.69,110.3699,106.02,108.73,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,100.0,81.890709,76.066911,66.666667,NaN,0,20.729694,94.77945,99.295547,88.054977
6411,2025-04-29,107.67,110.1999,107.44,109.02,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,100.0,81.890709,76.066911,66.666667,NaN,0,20.729694,94.77945,99.295547,88.054977
6412,2025-04-30,104.47,108.9200,104.08,108.92,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,100.0,81.890709,76.066911,66.666667,NaN,0,20.729694,94.77945,99.295547,88.054977
6413,2025-05-01,113.08,114.9400,111.30,111.61,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,100.0,81.890709,76.066911,66.666667,NaN,0,20.729694,94.77945,99.295547,88.054977
6414,2025-05-02,114.18,115.4000,113.37,114.50,12.111624,56.166891,0.406848,1.662900e+10,11.921894,...,100.0,81.890709,76.066911,66.666667,NaN,0,20.729694,94.77945,99.295547,88.054977


📈 Feature Engineering:
Try adding new ratios, rolling averages, or macroeconomic overlays.

Maybe the fundamental indicators alone don’t fully explain 7-day forward moves.

🔀 Try different targets:
Instead of 7-day forward return, try 3-day or 10-day.

Use volatility-adjusted thresholds (e.g., 1% move if the asset is volatile).

📊 Consider Ensemble:
Combine predictions or scores from Logistic + RF.

Or blend fundamental + technical scores.

# **Exporting the Fundamental File to drive **




In [201]:
fundamental_data_Close["Date"] = pd.to_datetime(fundamental_data_Close["Date"])

<ipython-input-201-c1997890364e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_Close["Date"] = pd.to_datetime(fundamental_data_Close["Date"])


In [202]:
five_years_ago = datetime.now() - timedelta(days=5*365)
fundamental_data_Close = fundamental_data_Close[fundamental_data_Close["Date"] >= five_years_ago]

In [203]:
fundamental_data_Close.head()

,Date,Open,High,Low,Close,RevenueGrowth,Profit_Margin,DebtToEquity,operatingCashflow,operatingExpenses,...,ROE Score,Op Expense Score,EPS Surprise Score,Piotroski_Score_Scaled,Forward_Return_7d,Target_Up,Final_Fundamental_Score_LogReg,Final_Fundamental_Score_RF,Final_Fundamental_Score_Corr,Fund_Score_comp
5161,2020-05-07,303.38,307.41,301.32,304.87,-0.805153,29.772727,0.77525,909000000.0,33.376623,...,35.002672,49.301091,85.714286,33.333333,0.148063,1,62.423911,41.295112,40.501336,52.166638
5162,2020-05-08,307.75,312.75,306.39,312.50,-0.805153,29.772727,0.77525,909000000.0,33.376623,...,35.002672,49.301091,85.714286,33.333333,0.127104,1,62.423911,41.295112,40.501336,52.166638
5163,2020-05-11,312.30,324.50,311.15,322.62,-0.805153,29.772727,0.77525,909000000.0,33.376623,...,35.002672,49.301091,85.714286,33.333333,0.112144,1,62.423911,41.295112,40.501336,52.166638
5164,2020-05-12,324.94,326.90,311.71,312.10,-0.805153,29.772727,0.77525,909000000.0,33.376623,...,35.002672,49.301091,85.714286,33.333333,0.124672,1,62.423911,41.295112,40.501336,52.166638
5165,2020-05-13,316.70,323.14,303.79,311.20,-0.805153,29.772727,0.77525,909000000.0,33.376623,...,35.002672,49.301091,85.714286,33.333333,0.160186,1,62.423911,41.295112,40.501336,52.166638


In [204]:
final_fundamental_score = fundamental_data_Close[[
    'Date', 'Open', 'High', 'Low', 'Close',
    'Revenue Score', "Profit Margin Score",
    "EPS Surprise Score", "Piotroski_Score_Scaled",
    'Fund_Score_comp'
]]

final_fundamental_score = final_fundamental_score.rename(columns={
    'Date': 'date',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Revenue Score': 'revenue score',
    'Fund_Score_comp': 'fundamental score'
})

final_fundamental_score["ticker"] = selected_ticker


In [205]:
final_fundamental_score = final_fundamental_score.round(2)

In [206]:
final_fundamental_score.head()

,date,open,high,low,close,revenue score,Profit Margin Score,EPS Surprise Score,Piotroski_Score_Scaled,fundamental score,ticker
5161,2020-05-07,303.38,307.41,301.32,304.87,22.78,75.95,85.71,33.33,52.17,NVDA
5162,2020-05-08,307.75,312.75,306.39,312.50,22.78,75.95,85.71,33.33,52.17,NVDA
5163,2020-05-11,312.30,324.50,311.15,322.62,22.78,75.95,85.71,33.33,52.17,NVDA
5164,2020-05-12,324.94,326.90,311.71,312.10,22.78,75.95,85.71,33.33,52.17,NVDA
5165,2020-05-13,316.70,323.14,303.79,311.20,22.78,75.95,85.71,33.33,52.17,NVDA


In [207]:
export_path = f"/content/drive/My Drive/StockDashboard_Automation/ExportToGitHub/fundamental_scores.csv"
final_fundamental_score.to_csv(export_path, index=False)

print(f"✅ Exported Fundamental Scores to {export_path}")

✅ Exported Fundamental Scores to /content/drive/My Drive/StockDashboard_Automation/ExportToGitHub/fundamental_scores.csv
